In [ ]:
import requests as r
from bs4 import BeautifulSoup
url = "https://cardfight.fandom.com/wiki"
data = r.get(f"{url}/Category:Cards")
soup = BeautifulSoup(data.text)
sets = []
def getCards(url):
    #print(url)
    data = r.get(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})
    soup = BeautifulSoup(data.text)
    #print(soup.prettify())
    #print(soup.find_all("a"))
    for x in soup.find_all("a"):
        try:
            #print(x['class'], x['class'] == ['category-page__member-link'])
            if x['class'] == ['category-page__member-link']:
                #print(x['href'])
                #print(x['href'].removeprefix('/wiki/'))
                name = x['href'].removeprefix('/wiki/')
                #print(name)
                if ("Card_Gallery:" not in name and "Category:" not in name and "Cardfight_Pack" not in name 
                    and "(ZERO)" not in name and "Set_Gallery:" not in name and "User:" not in name
                    and "List" not in name and "User blog:" not in name and "Forum:" not in name):
                    print(name)
                    sets.append(name)
                #print(name)
            if x['class'] == ['category-page__pagination-next', 'wds-button', 'wds-is-secondary']:
                print(x['href'])
                getCards(x['href'])
        except:
            pass
    print(sets)
getCards(f"{url}/Category:Cards")


In [ ]:
sets

In [ ]:
def generateSet(set, url = "https://cardfight.fandom.com/wiki"):
    import requests as r
    from bs4 import BeautifulSoup
    data = r.get(f"{url}/{set}")
    print(data.text)
    soup = BeautifulSoup(data.text)
    #print(soup.prettify())
    import pandas as pd
    df = pd.read_html(f"{url}/{set}")[0]
    code = df['Card No.'][0].split('/')[0]
    df.drop(["Card No.", "Clan", "Type", "Grade"], axis=1, inplace=True)
    return df, code
    #print(soup.find_all("table")[0])


In [ ]:
with open("cards.txt", "wb") as f:
    import pickle as p
    p.dump(sets, f)

In [ ]:
sets

In [ ]:
from Wikia import getWikiaCategory
getCards("https://cardfight.fandom.com/wiki/Category:Booster_Set")

In [ ]:
generateSet("Character_Deck_8:_Deck_of_Marine_Kings")

In [ ]:
from Wikia import getWikiaCategory
import pandas as pd
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session
def generateSet(set, url = "https://cardfight.fandom.com/wiki"):
    print(f"{url}/{set}")
    import requests as r
    a = r.get(f"{url}/{set}")
    if not a.ok:
        from time import sleep
        print("ERROR RETRYING")
        sleep(10)
        return generateSet(set, url)
    data = []
    try:
        df = pd.read_html(f"{url}/{set}")[0]
        #print(df)
    except:
        return [], "None"
    #print(df)
    try:
        code = df['Card No.'][0].split('/')[0]
        #print(code)
    except KeyError as e:
        #print(e)
        code = df['Code'][0].split('/')[0]
        #print(df)
        print("code")
    #print(code)
    try:
        data = df['Name'].to_list()
    except Exception as e:
        print(e)
        return [], "None"
    return data, code
    #print(soup.find_all("table")[0])
dct = {}
lst = (getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Booster_Set") 
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Extra_Booster")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Trial_Deck")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Technical_Booster")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Special_Series")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Character_Booster")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Clan_Booster")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Title_Booster")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Comic_Booster_Sets")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Mini_Booster")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Special_Sets")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Collector%27s_Set")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Trial_Deck")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Legend_Decks")
       + getWikiaCategory("https://cardfight.fandom.com/wiki/Category:Constructed_Deck")
)
#print(lst)
def setWrapper(dct, set):
    _, code = generateSet(set)
    while code in dct.keys():
        code += "#"
    dct[code] = set
    print(code)
    time.sleep(1)
import time
from threading import Thread
threads = []
for set in lst:
    if "List" not in set and "User" not in set:
        print(set)
        threads.append(Thread(target=setWrapper, args=(dct, set)))
        threads[-1].start()
        #print(code)
for thread in threads:
    thread.join()
print(dct)

In [ ]:
dct = {(code, set): dct[code, set] for code, set in dct.keys() if dct[code, set] != []}

In [ ]:
dct

In [ ]:
del dct

In [ ]:
import pickle as p
with open('SetsRaw.txt', 'wb') as f:
    p.dump(dct, f)

In [ ]:
getCards(f"{url}/Category:Booster_Set")

In [ ]:
print(dct)

In [ ]:
with open("SetRaw.txt", "w") as f:
    f.write(str(sets))

In [ ]:
with open("CardsRaw.txt", 'w', encoding='utf-8') as f:
    f.write(str(cards))

In [1]:
import pickle as p
with open("SetsRaw.txt", 'rb') as f:
    dct = p.load(f)
print(dct)

{'EB02': 'Extra_Booster:_Banquet_of_Divas', 'EB11': 'Extra_Booster:_Requiem_at_Dusk', 'D-CP01': 'D_Collector_Pack_01:_Genesis_of_the_Five_Greats', 'D-TB02': 'D_Title_Booster_02:_Monster_Strike', 'EB07': 'Extra_Booster:_Mystical_Magus', 'EB12': 'Extra_Booster:_Waltz_of_the_Goddess', 'EB05': 'Extra_Booster:_Celestial_Valkyries', 'EB10': 'Extra_Booster:_Divas_Duet', 'D-TB05': 'D_Title_Booster_05:_Record_of_Ragnarok', 'V-BT02': 'V_Booster_Set_02:_Strongest!_Team_AL4', 'BT04': 'Booster_Set_4:_Eclipse_of_Illusionary_Shadows', 'D-TB03': 'D_Title_Booster_03:_SHAMAN_KING_Vol.1', 'BT01': 'Booster_Set_1:_Descent_of_the_King_of_Knights', 'BT05': 'Booster_Set_5:_Awakening_of_Twin_Blades', 'V-BT07': 'V_Booster_Set_07:_Infinideity_Cradle', 'SW-BT01': 'SW_Booster_Set_1:_SWEAT16!_Iconic_Collection', 'BT02': 'Booster_Set_2:_Onslaught_of_Dragon_Souls', 'BT03': 'Booster_Set_3:_Demonic_Lord_Invasion', 'V-BT01': 'V_Booster_Set_01:_Unite!_Team_Q4', 'MBT01': 'Movie_Booster_1:_Neon_Messiah', 'D-BT05': 'D_Boost

In [11]:
import sqlite3 as sql
import pandas as pd
with sql.connect("cards.db") as conn:
    df = pd.read_sql("SELECT * FROM rarities", conn)

In [12]:
df

,id,name
0,1,RR
1,2,SSR
2,3,SP
3,4,SCR
4,5,C
...,...,...
79,80,URR
80,81,RLR
81,82,BSR
82,83,ЯЯЯ


In [22]:
for x in dct:
    print(x)
    print(dct[x])
    url = f"https://cardfight.fandom.com/wiki/{dct[x]}"
    print(url)
    import requests as r
    a = r.get(url)
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(a.text)
    #print(soup.prettify())
    b = [x for x in soup.find_all("li") if "cards" in x.text.lower() and "RR" in x.text][0].text.split("(")[1].split(")")[0].split(",")
    
    print(b)
    break

EB02
Extra_Booster:_Banquet_of_Divas
https://cardfight.fandom.com/wiki/Extra_Booster:_Banquet_of_Divas
['2 RRR', ' 5 RR', ' 8 R', ' 20 C + 2 SP.']


: 

In [ ]:
def generateCard(name: str, raw: bool = False, retries: int = 5):
    print(f"[%]{name}")
    if name == '-' or type(name) != str:
        print(f"[%]{name}")
        with open("ERR.txt", 'a') as f:
            f.write(f"{name}\n")
        return None
    import requests as r
    from time import sleep
    from bs4 import BeautifulSoup
    url = "https://cardfight.fandom.com/wiki"
    try:
        if raw:
            response = r.get(f'{url}/{name}', headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})
            gallery = r.get(f'{url}/Card_Gallery:{name}', headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})
        else:
            name = name.replace(" ", "_").replace(",", "%2C").replace("'", "%27").replace(":", "%3A").replace("!", "%21").replace("(", "%28").replace(")", "%29").replace("?", "%3F").replace(";", "%3B").replace("=", "%3D").replace("+", "%2B").replace("&", "%26").replace("/", "%2F").replace("#", "%23")
            response = r.get(f'{url}/{name.encode("utf-8").decode("utf-8")}', headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})
            gallery = r.get(f'{url}/Card_Gallery:{name.encode("utf-8").decode("utf-8")}', headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'})
    except Exception as e:
        print(f"ERR, {name}: {e}")
        if retries == 0:
            print("ERROR")
            with open("ERR.txt", 'a') as f:
                f.write(f"{name}\n")
            return None
        
        return generateCard(name, raw, retries - 1)
    #print(f'{url}/{name}')
    if not response.ok:
        
        print("[X]", response.status_code)
        print("[X]", name)
        print("ERROR RETRYING")
        sleep(10)
        if retries == 0:
            print("ERROR")
            with open("ERR.txt", 'a') as f:
                f.write(f"{name}\n")
            return None
        return generateCard(name, raw, retries - 1)
    soup = BeautifulSoup(response.text, 'html.parser')
    #print(response.url)
    dct = {'url': 'None', 'name': 'None', 'grade': -1, 'ability': 'None', 'power': 0, 'shield': 0, 'critical': 0, 
           'nation': ["None"], 'race': ['None'], 'format': 'None', 'type': "Normal Unit", 'trigger': ['None', 0], 'clan': ['None'], 
           'code': ['None'], 'skills': 'None',  'image': 'None', 'icon': 'None'}
    dct['url'] = response.url
    try:
        if soup.find_all('table')[0]['class'] == ["wikitable"]:
            value = 1
        else:
            value = 0
    except KeyError:
        value = 0
    except IndexError:
        print(f"ERR, {dct['url']}", "Nothing")
        return None
    try:
        lst = soup.find_all('table')[value]
    except IndexError:
        print(f"ERR, {dct['url']}", "Nothing")
        return None
    #print(lst)
    lst4 = []
    for div in soup.find_all('div'):
        try:
            #print(div["style"])
            if div["style"] == "float:left;":
                #print(div.find_all('a'))
                for a in div.find_all('a'):
                    try:
                        #print(a)
                        #print(a['class'])
                        if a['class'] == ['image']:
                            lst4.append(a['href'])
                    except:
                        pass
        except:
            pass
    #print(lst4)
    try:
        dct['image'] = lst4[0]
    except:
        print(f"ERR, {dct['url']}", "No Image")
        pass
    for x in lst.find_all('tr'):
        txt = x.text.strip().split('\n')
        if 'Name' in txt:
            dct['name'] = txt[1]
            #print(dct['name'])
        if 'Grade / Skill' in txt:
            dct['grade'] = txt[1].split('/')[0].strip().split(' ')[1]
            try:
                dct['ability'] = txt[1].split('/')[1].strip()
            except IndexError:
                dct['ability'] = "None"
        if 'Power' in txt:
            dct['power'] = txt[1].strip()
        if 'Shield' in txt:
            dct['shield'] = txt[1].strip()
        if 'Special Icon' in txt:
            dct['icon'] = txt[1].strip()
        if 'Imaginary Gift' in txt:
            #print(x)
            try:
                dct['icon'] = [y['title'] for y in x.find_all('a')][0].split('/')[1].strip()
            except:
                try:
                    dct['icon'] = txt[1].strip()
                except:
                    dct['icon'] = "None"
        if 'Critical' in txt:
            dct['critical'] = txt[1].strip()
        if 'Nation' in txt:
            dct['nation'] = [y.replace("'", ' ').strip() for y in txt[1].strip().split('/')]
        if 'Race' in txt:
            dct['race'] = [y.replace("'", ' ').strip() for y in txt[1].strip().split('/')]
        if 'Format' in txt:
            dct['format'] = [y.replace("'", ' ').strip() for y in txt[1].strip().split('/')]
            if dct['format'].count("Premium") > 1:
                dct['format'] = "V Premium"
            elif dct['format'].count("Standard") == 1:
                dct['format'] = "Standard"
            elif dct['format'].count("Premium") == 1:
                dct['format'] = "Premium"
            else:
                dct['format'] = "None"
        if 'Card Type' in txt:
            dct['type'] = txt[1].strip().split('/')[0]
        if 'Trigger Effect' in txt:
            try:
                dct['trigger'] = txt[1].strip().split('/')[0].replace("'", ' ').strip(), int(txt[1].strip().split('/')[1].split(',')[1].split('+')[1].strip())
            except IndexError:
                dct['trigger'] = txt[1].strip().split('/')[0].replace("'", ' ').strip(), int(txt[1].strip().split('/')[1].split('+')[1].strip())
        if 'Clan' in txt:
            #print([y.replace("'", ' ').strip() for y in txt[1].strip().split('/')])
            dct['clan'] = [y.replace("'", ' ').strip() for y in txt[1].strip().split('/')]
    try:
        lst1 = ''
        for i in range(1, 6):
            if "Set" in soup.find_all('table')[value + i].find_all('tr')[0].text.strip():
                #print(soup.find_all('table')[value + i].find_all('tr')[0].text.strip(), "#", soup.find_all('table')[value + i].find_all('tr')[1].text.strip())
                #print("A")
                lst1 = soup.find_all('table')[value + i].find_all('tr')[1]
                break
    except Exception as e:
        print(f"ERR, {dct['url']}: {e}")
        pass
    try:
        lst2 = ''
        for i in range(2, 6):
            if "Effect" in soup.find_all('table')[value + i].find_all('tr')[0].text.strip():
                #print(soup.find_all('table')[value + i].find_all('tr')[0].text.strip(), "#", soup.find_all('table')[value + i].find_all('tr')[1].text.strip())
                #print("B")
                lst2 = soup.find_all('table')[value + i].find_all('tr')[1]
                break
        #print(lst2)
        #print(lst3)
    
    except Exception as e:
        print(f"ERR, {dct['url']}: {e}")
        pass
    #print(lst2)
    #print(lst1[1].)
    #print(lst1[1].find_all('li'))
    #print(lst1, lst2)
    
    gallery = BeautifulSoup(gallery.text, 'html.parser')
    
    try:
        dct['code'] = []
        for x in soup.find_all('table', class_ = 'sets')[0].find_all('li'):
            try:
                if x.text.split('/')[0] == "PR":
                    dct['code'].append((x.text.split('/')[0], "Promo"))
                elif x.text.split('(')[1].split(')')[0] != "Sample":
                    dct['code'].append((x.text.split('/')[0].split(' - ')[1], x.text.split('(')[1].split(')')[0]))
            except Exception as e:
                try:
                    dct['code'].append((x.text.split('/')[0].split(' - ')[1], "C"))
                except Exception as e:
                    pass
                #print("Set Code: ", e)
                pass
        #print(gallery.find_all('div', class_ = "noarticletext mw-content-ltr"))
        try:
            for x in gallery.find_all('div', id = "gallery-0")[0].find_all('div', class_ = "lightbox-caption"):
                try:
                    if x.text.split('/')[0] == "PR":
                        dct['code'].append((x.text.split('/')[0], "Promo"))
                    elif x.text.split('(')[1].split(')')[0] != "Sample":
                        dct['code'].append((x.text.split('/')[0], x.text.split('(')[1].split(')')[0]))
                except Exception as e:
                    try:
                        dct['code'].append((x.text.split('/')[0], "C"))
                    except Exception as e:
                        pass
                    pass
        except Exception as e:
            pass
        dct['code'] = list({x for x in dct['code']})
        if dct['code'] == []:
            dct['code'] = "None"
    except Exception as e:
        print("[X]", e)
        dct['code'] = "None"
    try:
        if "Unrestricted" in lst2:
            lst2 = ''
        dct['skills'] = lst2
    except:
        dct['skills'] = "None"
    if 'trigger' not in dct.keys():
        dct['trigger'] = "None"
    if 'type' not in dct.keys():
        dct['type'] = "Normal Unit"
    if 'shield' not in dct.keys():
        dct['shield'] = 0
    if 'clan' not in dct.keys():
        dct['clan'] = ["None"]
    if 'power' not in dct.keys():
        dct['power'] = 0
    if 'nation' not in dct.keys():
        dct['nation'] = ["None"]
    if 'race' not in dct.keys():
        dct['race'] = "None"
    if 'critical' not in dct.keys():
        dct['critical'] = 0
    if 'format' not in dct.keys():
        dct['format'] = "None"
    if 'ability' not in dct.keys():
        dct['ability'] = "None"
    if 'grade' not in dct.keys():
        dct['grade'] = "None"
    if 'icon' not in dct.keys():
        dct['icon'] = "None"
    #print(soup.find_all('table')[value + 3].find_all('tr')[1].text)
    card = dct

    return card
print(generateCard('Steam Gunner, Yarlaganda'))
print(generateCard('Spirit of Sword, Yoh Asakura'))
print(generateCard('Yamata no Orochi Go, Wooden Sword Ryu'))

In [ ]:
import requests as r
from bs4 import BeautifulSoup
a = r.get("https://cardfight.fandom.com/wiki/Chronojet_Dragon_(V_Series)")
soup = BeautifulSoup(a.text)

for x in soup.find_all('table', class_ = 'sets')[0].find_all('li'):
    try:
        print(x.text.split('/')[0].split(' - ')[1], x.text.split('(')[1].split(')')[0])
    except Exception as e:
        print(e)

In [ ]:
names = []
lst = []

In [ ]:


import time
from IPython.display import display, HTML
from threading import Thread
lst = []
def getCards(names, lst):
    for name in names:
        lst.append(generateCard(name))

threads = []
N = 8
print(len(sets))
print(sets)
piece = len(sets) // N
for i in range(N):
    print(f"Thread {i} Starts [{sets[piece * i]} - {sets[piece * (i + 1)]}]")
    threads.append(Thread(target=getCards, args=(sets[piece * i: piece * (i + 1)], lst)))
    threads[-1].start()
print(f"Thread {i} Starts [{sets[piece * N]} - {sets[-1]}]")
threads.append(Thread(target=getCards, args=(sets[piece * N:], lst)))
threads[-1].start()
for x, thread in enumerate(threads):
    print(f"Thread {x} Ends")
    thread.join()
print(lst)

In [ ]:
import pandas as pd
while None in lst:
    lst.remove(None)
df = pd.DataFrame(lst).sort_values('name')
df.to_csv("cards.csv")

In [ ]:
df[df["name"] == "Hades Dragon Deity of Resentment, Gallmageheld"]

In [1]:
import pandas as pd
from Cards import *
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm import tqdm
try:
    os.remove("cards.db")
except:
    pass
df = pd.read_csv("cards.csv")
print(df)
import pickle as p
with open("SetsRaw.txt", 'rb') as f:
    dct = p.load(f)
print(dct)
cardEngine = create_engine(r"sqlite:///cards.db")

       Unnamed: 0                                                url  \
0               5  https://cardfight.fandom.com/wiki/%22Assassin%...   
1              12  https://cardfight.fandom.com/wiki/%22Awakened_...   
2              21  https://cardfight.fandom.com/wiki/%22Bloom_of_...   
3              29  https://cardfight.fandom.com/wiki/%22Goma_Chob...   
4              39  https://cardfight.fandom.com/wiki/%22I_Introdu...   
...           ...                                                ...   
12802          16  https://cardfight.fandom.com/wiki/Zoomdive_Dra...   
12803          26  https://cardfight.fandom.com/wiki/Zubat_Battle...   
12804          34  https://cardfight.fandom.com/wiki/Zypsophilia_...   
12805       12438         https://cardfight.fandom.com/wiki/Zbing_z.   
12806          42  https://cardfight.fandom.com/wiki/%E2%9C%A9Daz...   

                                                    name  grade       ability  \
0                 "Assassin" of the Mino Province, Kic

In [10]:
print(df[df["shield"] == "15000+"]["name"])

7469    Medical Studies of Extinction, Vincent
Name: name, dtype: object


In [2]:
triggers = {}
types = {}
cards = {}
clans = {}
nations = {}
races = {}
sets = {}
rarities = {}
Base.metadata.create_all(cardEngine)
count = 0
lenght = len(df)
with Session(cardEngine) as conn:
    for x in conn.execute(select(Card)).all():
        cards[x.name] = x
    for x in conn.execute(select(Trigger)).all():
        triggers[x.name] = x
    for x in conn.execute(select(Type)).all():
        types[x.name] = x
    for x in conn.execute(select(Clan)).all():
        clans[x.name] = x
    for x in conn.execute(select(Nation)).all():
        nations[x.name] = x
    for x in conn.execute(select(Race)).all():
        races[x.name] = x
    for x in conn.execute(select(Set)).all():
        sets[x.code] = x
    for x in conn.execute(select(Rarity)).all():
        rarities[x.name] = x
    for x in tqdm(df.index):
        try:
            name = str(df.iloc[x]["name"]) if str(df.iloc[x]["name"]) != 'nan' else None
            if name in cards.keys():
                continue
            #print(df.iloc[x])
            trigger = tuple(eval(''.join(list(df.iloc[x]["trigger"]))))
            type = df.iloc[x]["type"]
            if trigger not in triggers.keys() and trigger[0] != 'None':
                triggers[trigger] = Trigger(trigger[0], trigger[1])
                conn.add(triggers[trigger])
            else:
                if trigger[0] == 'None':
                    trigger = None
            if type not in types.keys():
                types[type] = Type(type)
                conn.add(types[type])
            conn.flush()
            #print(trigger)
            #print(types[type])
            url = str(df.iloc[x]["url"]) if str(df.iloc[x]["url"]) != 'nan' else None
            image = str(df.iloc[x]["image"]) if str(df.iloc[x]["image"]) != 'nan' else None
            skills = str(''.join([x for x in df.iloc[x]["skills"]])) if str(df.iloc[x]["skills"]) != 'nan' else None
            grade = int(df.iloc[x]["grade"]) if str(df.iloc[x]["grade"]) != 'nan' else None
            ability = str(df.iloc[x]["ability"]) if str(df.iloc[x]["ability"]) != 'nan' else None
            power = str(df.iloc[x]["power"]) if str(df.iloc[x]["power"]) != 'nan' else 0
            shield = int(df.iloc[x]["shield"].replace(',','').replace('.','')) if str(df.iloc[x]["shield"]) != 'nan' else 0
            critical = int(df.iloc[x]["critical"]) if str(df.iloc[x]["critical"]) != 'nan' else 0
            format = str(df.iloc[x]["format"]) if str(df.iloc[x]["format"]) != 'nan' else None
            icon = str(df.iloc[x]["icon"]) if str(df.iloc[x]["icon"]) != 'nan' else None
            card = Card(name=name, url=url, image=image, skills=skills, grade=grade, ability=ability, power=power, shield=shield, critical=critical, trigger=triggers[trigger].id if trigger is not None else None, type=types[type].id, format=format, icon=icon)
            cards[name] = card
            conn.add(card)
            conn.flush()
            #print(card)
            nation = eval(''.join(list(df.iloc[x]["nation"])))
            race = eval(''.join(list(df.iloc[x]["race"])))
            clan = eval(''.join(list(df.iloc[x]["clan"])))
            #print(name, str(df.iloc[x]["code"]) == 'nan')
            rarityset = eval(''.join(list(df.iloc[x]["code"]))) if str(df.iloc[x]["code"]) != 'nan' else []
            #print(rarityset)

            #print(nation, race, clan, rarityset)
            for i in nation:
                if i not in nations.keys():
                    nations[i] = Nation(i)
                    conn.add(nations[i])
                conn.flush()
                #print(nations[i])
                cardnation = CardNation(card=card.id, nation=nations[i].id)
                #print(cardnation)
                conn.add(cardnation)
                conn.flush()
            for i in race:
                if i not in races.keys():
                    races[i] = Race(i)
                    conn.add(races[i])
                #print(races[i])
                conn.flush()
                #print(races[i])
                cardrace = CardRace(card=card.id, race=races[i].id)
                #print(cardrace)
                conn.add(cardrace)
            for i in clan:
                if i not in clans.keys():
                    clans[i] = Clan(i)
                    conn.add(clans[i])
                conn.flush()
                #print(clans[i])
                cardclan = CardClan(card=card.id, clan=clans[i].id)
                #print(cardclan)
                conn.add(cardclan)
            for set, rarity in rarityset:
                if set not in sets.keys():
                    if set in dct.keys():
                        #print(set, dct[set])
                        sets[set] = Set(set, dct[set])
                    else:
                        sets[set] = Set(set)
                    conn.add(sets[set])
                if rarity not in rarities.keys():
                    rarities[rarity] = Rarity(rarity)
                    conn.add(rarities[rarity])
                conn.flush()
                #print(sets[set])
                #print(rarities[rarity])
                cardsetrarity = CardSetRarity(card=card.id, set=sets[set].id, rarity=rarities[rarity].id)
                conn.add(cardsetrarity)
                #print(cardsetrarity)
            count += 1
            conn.commit()
            #break
        except Exception as e:
            print(df.iloc[x])
            raise e
    conn.commit()

100%|██████████| 12807/12807 [38:11<00:00,  5.59it/s] 


In [ ]:
import os
df = None
for x in os.listdir():
    if ".csv" in x:
        if df is None:
            df = pd.read_csv(x)
        else:
            df = pd.concat([df, pd.read_csv(x)])
df.to_csv("Cards.csv")

In [ ]:
df.to_csv("Cards.csv")

In [ ]:
df['name'].to_list().count('Chronojet Dragon Z')   

In [ ]:
df["name"].to_list().count('Chronofang Tiger')

In [ ]:
df.to_csv("Cards.csv")

In [ ]:
lst[-4]

In [ ]:
nations = {}
clans   = {}
races   = {}
sets    = {}
from sqlalchemy import Engine, create_engine
def store(dct: dict, engine: Engine):
    print(dct)
    from Cards import Base, Card, Nation, Clan, Set, Race, Trigger, createTable
    if dct == None:
        return None
    try:
        Base.metadata.create_all(engine)
    except:
        pass
    from sqlalchemy.orm import Session
    with Session(engine) as session:
        if session.query(Trigger).where(Trigger.name == dct['trigger'][0]).where(Trigger.power == dct['trigger'][1]).first() is None:
            trigger = Trigger(dct['trigger'][0], dct['trigger'][1])
        else:
            trigger = session.query(Trigger).where(Trigger.name == dct['trigger'][0]).where(Trigger.power == dct['trigger'][1]).first()
        session.add(trigger)
        session.commit()
        card = Card(
            name = dct['name'],
            grade = dct['grade'],
            ability = dct['ability'],
            power = dct['power'],
            shield = dct['shield'],
            critical = dct['critical'],
            specialIcon = dct['icon'],
            skills = dct['skills'],
            cardType = dct['type'],
            image = dct['image'],
            format = dct['format'],
            trigger = trigger.triggerID,
            url = dct['url']
        )
        session.add(card)
        session.commit()
        lst = []
        for x in set(dct['nation']):
            #print(x, nations.keys())
            if x not in nations.keys():
                nations[x] = createTable(x, "Nation")
                lst.append(Nation(x))
            lst.append(nations[x](card.cardID))
        for x in set(dct['clan']):
            #print(x, clans.keys())
            if x not in clans.keys():
                clans[x] = createTable(x, "Clan")
                lst.append(Clan(x))
            lst.append(clans[x](card.cardID))
        for x in set(dct['race']):
            #print(x, races.keys())
            if x not in races.keys():
                races[x] = createTable(x, "Race")
                lst.append(Race(x))
            lst.append(races[x](card.cardID))
        for x in set(dct['code']):
            #print(x, sets.keys())
            if x not in sets.keys():
                sets[x] = createTable(x, "Set")
                session.add(Set(x))
            lst.append(sets[x](card.cardID))
        Base.metadata.create_all(engine)
        session.add_all(lst)
        session.commit()


In [ ]:
lst

In [ ]:
engine = create_engine(r"sqlite:///D:\Projects\CardfightBot\cards.db")

In [ ]:
cards[-1]

In [ ]:
from threading import Thread
import time as t
dct = []
count = 0 
for card in cards:
    count+=1
    print(count)
    store(generateCard(card, True), engine)

In [ ]:
for x in lst:
    print(x)

In [ ]:
import sqlite3 as sql 
conn = sql.connect("cards.db")
c = conn.cursor()
c.execute("CREATE TABLE if not exists cards (name TEXT, grade INT, clan TEXT, type TEXT, power INT, shield INT, critical INT, nation TEXT, clan TEXT, skills TEXT, race TEXT, format TEXT, trigger TEXT)")
for x in lst:
    x = x.toDict()
    c.execute("INSERT INTO cards VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", (x['name'], x['grade'], x['clan'], x['type'], x['power'], x['shield'], x['critical'], x['nation'], x['clan'], x['skills'], x['race'], x['format'], x['trigger']))
    for y in x['sets']:
        c.execute(f"create table if not exists {y} (name TEXT))")
        c.execute(f"insert into {y} values (?)", (x['name'],))
    conn.commit()

In [ ]:
from Cards import createSet, Set
a,set1 = createSet("V-SS08"), Set("V-SS08")
b,set2 = createSet("V-SS07"), Set("V-SS07")

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
engine = create_engine(r"sqlite:///D:\Projects\CardfightBot\A.db", echo=True)
with Session(engine) as session:
    a.metadata.create_all(engine)
    session.add_all([set1, set2])
    session.add_all([a(1), a(2), b(1), b(2)])
    session.commit()

In [ ]:
from Cards import Set, Race, Nation, Clan, Trigger, Card, createTable
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
engine = create_engine(r"sqlite:///cards.db")
with Session(engine) as session:
    c = {x.name: [session.query(Card).where(Card.cardID == y.cardID).all() for y in session.query(createTable(x.name, "Set")).all()] for x in session.query(Set).all()}

In [ ]:
name = df["name"]
df[name.isin(name[name.duplicated()])].sort_values("name")

In [ ]:
df1 = df.drop_duplicates(subset = "name", keep='first')

In [ ]:
df1

In [ ]:
import sqlite3 as sql
with sql.connect("Backup/cards.db") as conn:
    df2 = pd.read_sql("SELECT * FROM cards", conn)
from IPython import display

display.display(df1[~df1["name"].isin(df2["name"])]["name"])
display.display(df2[~df2["name"].isin(df1["name"])]["name"].to_list())

In [ ]:
df2